# Intro. to Snorkel: Extracting Spouse Relations from the News

## Part III: Training an End Extraction Model

In this final section of the tutorial, we'll use the noisy training labels we generated in the last tutorial part to train our end extraction model.

For this tutorial, we will be training a Bi-LSTM, a state-of-the-art deep neural network implemented in [TensorFlow](https://www.tensorflow.org/).

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os

# TO USE A DATABASE OTHER THAN SQLITE, USE THIS LINE
# Note that this is necessary for parallel execution amongst other things...
# os.environ['SNORKELDB'] = 'postgres:///snorkel-intro'

from snorkel import SnorkelSession
session = SnorkelSession()

We repeat our definition of the `Spouse` `Candidate` subclass:

In [2]:
from snorkel.models import candidate_subclass

Spouse = candidate_subclass('Spouse', ['person1', 'person2'])

In [3]:
import os
os.environ['PATH']

'/lfs/1/jdunnmon/repos/anaconda3/envs/metal/bin:/lfs/1/jdunnmon/repos/anaconda3/envs/metal/bin:/lfs/local/0/jdunnmon/repos/basset/src:/lfs/local/0/jdunnmon/repos/torch/bin:/dfs/scratch1/jdunnmon/repos/spark-2.3.1-bin-hadoop2.7/bin:/lfs/1/jdunnmon/repos/anaconda3/bin:/lfs/local/0/jdunnmon/repos/anaconda3/bin:/lfs/local/0/jdunnmon/repos/basset/src:/lfs/local/0/jdunnmon/repos/torch/bin:/dfs/scratch1/jdunnmon/repos/spark-2.3.1-bin-hadoop2.7/bin:/lfs/1/jdunnmon/repos/anaconda3/bin:/lfs/local/0/jdunnmon/repos/anaconda3/bin:/lfs/local/0/jdunnmon/repos/basset/src:/lfs/local/0/jdunnmon/repos/torch/bin:/dfs/scratch1/jdunnmon/repos/spark-2.3.1-bin-hadoop2.7/bin:/lfs/1/jdunnmon/repos/anaconda3/bin:/lfs/local/0/jdunnmon/repos/anaconda3/bin:/usr/kerberos/sbin:/usr/kerberos/bin:/afs/cs/software/sbin:/afs/cs/software/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/usr/local/cuda/bin:/snap/bin'

We reload the probabilistic training labels:

In [4]:
from snorkel.annotations import load_marginals

train_marginals = load_marginals(session, split=0)

We also reload the candidates:

In [5]:
train_cands = session.query(Spouse).filter(Spouse.split == 0).order_by(Spouse.id).all()
dev_cands   = session.query(Spouse).filter(Spouse.split == 1).order_by(Spouse.id).all()
test_cands  = session.query(Spouse).filter(Spouse.split == 2).order_by(Spouse.id).all()

Finally, we load gold labels for evaluation:

In [6]:
from snorkel.annotations import load_gold_labels

L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1)
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)

Now we can setup our discriminative model. Here we specify the model and learning hyperparameters.

They can also be set automatically using a search based on the dev set with a [GridSearch](https://github.com/HazyResearch/snorkel/blob/master/snorkel/learning/utils.py) object.

In [9]:
from snorkel.learning.pytorch import LSTM

train_kwargs = {
    'lr':            0.01,
    'embedding_dim': 50,
    'hidden_dim':    50,
    'n_epochs':      10,
    'dropout':       0.25,
    'seed':          1701,
    'batch_size':    256,
}

lstm = LSTM(n_threads=None, gpu=True).cuda()
lstm.train(train_cands, train_marginals, X_dev=dev_cands, Y_dev=L_gold_dev, **train_kwargs)

[LSTM] Training model
[LSTM] n_train=16066  #epochs=10  batch size=256
[LSTM] Epoch 1 (32.82s)	Average loss=0.617586	Dev F1=0.00
[LSTM] Epoch 2 (70.65s)	Average loss=0.598171	Dev F1=23.76
[LSTM] Epoch 3 (108.52s)	Average loss=0.591410	Dev F1=22.31
[LSTM] Epoch 4 (146.52s)	Average loss=0.588148	Dev F1=21.92
[LSTM] Epoch 5 (184.44s)	Average loss=0.585072	Dev F1=29.54
[LSTM] Epoch 6 (222.44s)	Average loss=0.583089	Dev F1=24.06
[LSTM] Epoch 7 (260.87s)	Average loss=0.581405	Dev F1=34.75
[LSTM] Epoch 8 (298.75s)	Average loss=0.581254	Dev F1=26.29
[LSTM] Epoch 9 (336.58s)	Average loss=0.578787	Dev F1=23.93
[LSTM] Model saved as <LSTM>
[LSTM] Epoch 10 (374.67s)	Average loss=0.577583	Dev F1=26.09
[LSTM] Model saved as <LSTM>
[LSTM] Training done (379.68s)
[LSTM] Loaded model <LSTM>


Now, we get the precision, recall, and F1 score from the discriminative model:

In [10]:
p, r, f1 = lstm.score(test_cands, L_gold_test)
print("Prec: {0:.3f}, Recall: {1:.3f}, F1 Score: {2:.3f}".format(p, r, f1))

Prec: 0.266, Recall: 0.682, F1 Score: 0.383


We can also get the candidates returned in sets (true positives, false positives, true negatives, false negatives) as well as a more detailed score report:

In [11]:
tp, fp, tn, fn = lstm.error_analysis(session, test_cands, L_gold_test)

Scores (Un-adjusted)
Pos. class accuracy: 0.682
Neg. class accuracy: 0.91
Precision            0.266
Recall               0.682
F1                   0.383
----------------------------------------
TP: 75 | FP: 207 | TN: 2102 | FN: 35



Note that if this is the final test set that you will be reporting final numbers on, to avoid biasing results you should not inspect results.  However you can run the model on your _development set_ and, as we did in the previous part with the generative labeling function model, inspect examples to do error analysis.

You can also improve performance substantially by increasing the number of training epochs!

Finally, we can save the predictions of the model on the test set back to the database. (This also works for other candidate sets, such as unlabeled candidates.)

In [12]:
lstm.save_marginals(session, test_cands)

Saved 2419 marginals


##### More importantly, you completed the introduction to Snorkel! Give yourself a pat on the back!